In [99]:
import sys
import os
import tensorflow as tf
import pandas as pd
import numpy as np
from   sklearn.preprocessing import LabelBinarizer
from   sklearn.preprocessing import LabelEncoder
from   sklearn.preprocessing import Imputer
import numpy 
from   collections import Counter


from IPython.core.display import display


columns = ["PassengerId", "Survived", "Pclass", "Name", 
           "Sex", "Age", "SibSp", "Parch", 
           "Ticket", "Fare", "Cabin", "Embarked"]

#Same as columns but with "Survived" removed
features =  ["PassengerId", "Pclass", "Name", 
           "Sex", "Age", "SibSp", "Parch", 
           "Ticket", "Fare", "Cabin", "Embarked"]

labels   = 'Survived'

cat_features = ["Pclass","Sex","Embarked"]

num_features   = ["Age","SibSp","Parch","Fare"]

useless_features = ["PassengerId", "Name","Ticket","Cabin"]

labels   = "Survived"

def main():
    data_set       = pd.read_csv("train.csv")                #download training/training set
    validation_set = pd.read_csv("test.csv")
    data_set       = data_set.drop(useless_features, axis=1) #drop useless features
    data_set       = data_set.dropna()                       #drop rows with missing data         
    training_set, test_set = split_frame(data_set)
    
    training_features, training_labels = training_set, training_set.pop(labels)
    test_features, test_labels = test_set, test_set.pop(labels)
    
    classifier        = model(feature_columns(data_set))
    
    training_features = df_to_dict_list(training_features)
    test_features = df_to_dict_list(test_features)
    classifier.train(
        input_fn=lambda:training_input_fn(training_features, training_labels,50),
        steps=2000)
    eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_features, test_labels, 100))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features, labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
    
def unique_val(data_set,feature_name):
    my_list=[]
    for i in set(data_set[feature_name].tolist()):
        my_list.append(i)
    return my_list

def training_input_fn(features, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(buffer_size=100).repeat(count=None).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

def df_to_dict_list(data_set):
    my_list={}
    for column in data_set.columns.values:
        my_list[column] = data_set[column].tolist()
    return my_list

def feature_columns(data_set, cat_features=cat_features, \
                    num_features=num_features):
    
    numeric_feature_column = []
    for feature in num_features:
        numeric_feature_column.append(tf.feature_column.numeric_column(
            key=feature,
            dtype=tf.float32))
    
    categorical_feature_column = []
    for feature in cat_features:
        categorical_feature_column.append(tf.feature_column.categorical_column_with_vocabulary_list(
            key=feature,
            vocabulary_list=unique_val(data_set,feature)
        ))
    
    indicator_columns=[]
    for column in categorical_feature_column:
        indicator_columns.append(tf.feature_column.indicator_column(column))
    feature_columns = numeric_feature_column  + indicator_columns
    print(feature_columns)
  
    return feature_columns


def encode_columns(data_set, cat_features=cat_features):
    for feature in cat_features:
        label_encoder = LabelEncoder()
        label_binarizer = LabelBinarizer()
        
        label_encoder.fit(data_set[feature].tolist())
        encoded_train = label_encoder.transform(data_set[feature].tolist())
        n_train = data_set.columns[data_set.columns.get_loc(feature)]

        data_set.drop(n_train, axis=1, inplace=True)
        label_binarizer.fit(encoded_train.tolist())
        data_set[n_train] = label_binarizer.transform(encoded_train.tolist()).tolist()

    

def model(feature_columns):
    return tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[10,10],
    optimizer=tf.train.FtrlOptimizer(
        learning_rate=0.1,
        l1_regularization_strength=1.0,
        l2_regularization_strength=1.0),
        activation_fn=tf.nn.relu,
    n_classes=2,
    )

def split_frame(data_set):
    mask = np.random.rand(len(data_set)) < 0.8
    training_set = data_set[mask]
    test_set     = data_set[~mask]
    return training_set, test_set
    
    
main()




[_NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='SibSp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='Parch', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='Fare', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='Pclass', vocabulary_list=(1, 2, 3), dtype=tf.int64, default_value=-1, num_oov_buckets=0)), _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='Sex', vocabulary_list=('female', 'male'), dtype=tf.string, default_value=-1, num_oov_buckets=0)), _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='Embarked', vocabulary_list=('S', 'Q', 'C'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmphdr